In [1]:
import pickle,os
from urllib.request import urlopen
import numpy as np

In [19]:
cwd = os.getcwd()
data_folder_path = os.path.join(cwd+"/sample-Ising")
#cwd = 0
#data_folder_path = 0

In [3]:
def directory_list(target_path):
    path_list = []
    for name in os.listdir(target_path):
        if(name=="Ising2DFM_reSample_L40_T=All_labels.pkl"):
            continue
        path = os.path.join(target_path, name)
        path_list.append(path)
    return path_list

In [20]:
sample_path_list = directory_list(data_folder_path)
#sample_path_list = 0

In [5]:
sample = np.zeros((10000,1600))
sample_label = np.zeros(10000,dtype='i8')
for i in range(8):
    with open(sample_path_list[2*i+1],mode="rb") as f:
        hoge = pickle.load(f)
        data = np.unpackbits(hoge).reshape(-1,1600)
        sample = np.concatenate((sample,data))
        sample_label = np.concatenate((sample_label,np.array([i for j in range(10000)])))

In [6]:
print(sample.shape)
sample=sample[10000:]
print(sample.shape)

(90000, 1600)
(80000, 1600)


In [7]:
sample_label = sample_label[10000:]

In [8]:
sample_label[0],sample_label[10000],sample_label[20000]

(0, 1, 2)

In [9]:
def divide_train_test(data,label, test_ratio=0.3):
    shuffled = np.random.permutation(len(data))
    test_size = int(len(data)*test_ratio)
    test_index = shuffled[:test_size]
    train_index = shuffled[test_size:]
    return data[train_index],label[train_index],data[test_index],label[test_index]

In [10]:
x_train,y_train,x_test,y_test = divide_train_test(sample,sample_label)

ノートPC(メモリ8GB)だと容量がきついため？（ここで`sample`,`sample_label`を解放しておかないと訓練セットをつくるときにカーネルが落ちる)

In [11]:
sample = 0
sample_label = 0

In [12]:
from keras.utils import to_categorical
from keras.layers import Input,Dense
from keras.models import Model
from keras.optimizers import Adam
opt = Adam(lr=0.0005)

Using TensorFlow backend.


In [13]:
NUM_CLASSES = 8
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = to_categorical(y_train,NUM_CLASSES)
y_test = to_categorical(y_test,NUM_CLASSES)

In [15]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((56000, 1600), (24000, 1600), (56000, 8), (24000, 8))

In [14]:
input_layer = Input(shape=(1600,))
x = Dense(units=200,activation='relu')(input_layer)
x = Dense(units=100,activation='relu')(x)
x = Dense(units=50,activation='relu')(x)
outpu_layer = Dense(units=8,activation='softmax')(x)
model = Model(input_layer,outpu_layer)

In [16]:
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [17]:
model.fit(x_train,y_train,batch_size=32,epochs=10,shuffle=True)

Epoch 1/10
56000/56000 [==============================] - 8s 148us/step - loss: 1.1388 - accuracy: 0.4737
Epoch 2/10
56000/56000 [==============================] - 8s 147us/step - loss: 0.9015 - accuracy: 0.5578
Epoch 3/10
56000/56000 [==============================] - 8s 148us/step - loss: 0.8211 - accuracy: 0.5938
Epoch 4/10
56000/56000 [==============================] - 8s 149us/step - loss: 0.7719 - accuracy: 0.6149
Epoch 5/10
56000/56000 [==============================] - 8s 149us/step - loss: 0.7207 - accuracy: 0.6384
Epoch 6/10
56000/56000 [==============================] - 8s 150us/step - loss: 0.7033 - accuracy: 0.6472
Epoch 7/10
56000/56000 [==============================] - 9s 153us/step - loss: 0.6700 - accuracy: 0.6615
Epoch 8/10
56000/56000 [==============================] - 9s 154us/step - loss: 0.6456 - accuracy: 0.6755
Epoch 9/10
56000/56000 [==============================] - 8s 150us/step - loss: 0.6203 - accuracy: 0.6864
Epoch 10/10
56000/56000 [=====================

In [18]:
model.evaluate(x_test,y_test)

24000/24000 [==============================] - 1s 35us/step


[0.8353426659504573, 0.6139583587646484]